In [1]:
# ============================
# Cell 1: Core Imports and Response Structure
# ============================
import os
import time
import json
import requests
import logging
import pickle
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from concurrent.futures import ThreadPoolExecutor, TimeoutError as FutureTimeoutError
from collections import defaultdict

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class TherapeuticResponse:
    """Enhanced response structure for therapeutic context"""
    text: str
    timestamp: float
    error: bool = False
    processing_time: float = 0.0
    error_details: str = ""
    timeout: bool = False
    empathy_score: float = 0.0
    safety_checks: List[str] = None
    ethical_considerations: List[str] = None
    refinement_suggestions: List[str] = None
    crisis_flag: bool = False

# ============================
# Cell 2: Ollama Client Implementation
# ============================
class OllamaClient:
    """Robust Ollama client with configurable timeouts"""
    def __init__(self, model_name: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L", base_url: str = "http://localhost:11434"):
        self.model_name = model_name
        self.base_url = base_url
        self.max_retries = 5
        self.request_timeout = 300
        self._verify_model()

    def _parse_json_safe(self, text: str):
        clean_text = text.strip()
        if not clean_text:
            return {"error": "Empty response"}
        try:
            return json.loads(clean_text)
        except json.JSONDecodeError:
            try:
                start = clean_text.find('{')
                end = clean_text.rfind('}') + 1
                return json.loads(clean_text[start:end])
            except Exception:
                return {"error": f"Invalid JSON format: {clean_text[:200]}..."}
        except Exception as e:
            return {"error": str(e)}

    def _verify_model(self):
        """Verify that the model is available; pull it if necessary."""
        for attempt in range(self.max_retries):
            try:
                resp = requests.get(f"{self.base_url}/api/tags", timeout=10)
                if resp.status_code == 200:
                    data = self._parse_json_safe(resp.text)
                    models = [m['name'] for m in data.get('models', [])]
                    if any(self.model_name in m for m in models):
                        return
                    self._pull_model()
                    return
                logger.warning(f"Model check failed (status {resp.status_code})")
            except Exception as e:
                logger.warning(f"Model check attempt {attempt+1} failed: {e}")
                time.sleep(2 ** attempt)
        raise ConnectionError(f"Couldn't connect to Ollama after {self.max_retries} attempts")

    def _pull_model(self):
        """Pull model with progress tracking."""
        try:
            resp = requests.post(
                f"{self.base_url}/api/pull",
                json={"name": self.model_name},
                stream=True,
                timeout=600
            )
            for line in resp.iter_lines():
                if line:
                    try:
                        status = self._parse_json_safe(line).get('status', '')
                        logger.info(f"Pull progress: {status}")
                    except Exception:
                        continue
        except Exception as e:
            logger.error(f"Model pull failed: {e}")
            raise

    def generate(self, prompt: str) -> Tuple[str, bool]:
        """Generate a response from the language model with retries."""
        for attempt in range(self.max_retries):
            try:
                with ThreadPoolExecutor() as executor:
                    future = executor.submit(
                        requests.post,
                        f"{self.base_url}/api/generate",
                        json={
                            "model": self.model_name,
                            "prompt": prompt[:4000],
                            "stream": False,
                            "options": {"temperature": 0.5}
                        },
                        timeout=self.request_timeout
                    )
                    resp = future.result(timeout=self.request_timeout)
                    data = self._parse_json_safe(resp.text)
                    return data.get("response", ""), False
            except FutureTimeoutError:
                logger.warning(f"Generation timed out (attempt {attempt+1})")
                return f"Error: Timeout after {self.request_timeout}s", True
            except Exception as e:
                logger.warning(f"Attempt {attempt+1} failed: {e}")
                time.sleep(1)
        return f"Error: Failed after {self.max_retries} attempts", True

# ============================
# Cell 3: Base Agent Framework
# ============================
class BaseAgent:
    """Timeout-aware base agent that utilizes the Ollama client."""
    def __init__(self, client: OllamaClient):
        self.client = client
        self.retry_count = 3
        self.max_wait = 300
        
    def safe_generate(self, prompt: str) -> TherapeuticResponse:
        start_time = time.time()
        timeout_occurred = False
        if not isinstance(prompt, str) or len(prompt.strip()) == 0:
            return TherapeuticResponse(
                text="Error: Invalid input prompt",
                timestamp=start_time,
                error=True,
                error_details="Empty or non-string prompt",
                processing_time=0.0
            )
        for attempt in range(self.retry_count):
            try:
                with ThreadPoolExecutor() as executor:
                    future = executor.submit(self.client.generate, prompt)
                    text, error = future.result(timeout=self.max_wait)
                    return TherapeuticResponse(
                        text=text,
                        timestamp=start_time,
                        error=error,
                        processing_time=time.time() - start_time,
                        error_details=text if error else "",
                        timeout=timeout_occurred
                    )
            except FutureTimeoutError:
                logger.error(f"Generation timed out after {self.max_wait}s")
                timeout_occurred = True
            except Exception as e:
                error_msg = str(e)
                logger.error(f"Generation error: {e}")
        return TherapeuticResponse(
            text=f"Final error: {error_msg}" if 'error_msg' in locals() else "Unknown error",
            timestamp=start_time,
            error=True,
            error_details=error_msg if 'error_msg' in locals() else "",
            processing_time=time.time() - start_time,
            timeout=timeout_occurred
        )

# ============================
# Cell 4: Intimacy Agents Implementation
# ============================
class IntimacyContextAnalyzer(BaseAgent):
    """Analyzes intimacy needs and communication patterns."""
    def analyze_desires(self, input_text: str) -> Dict:
        prompt = f"""Analyze intimacy context (sex-positive focus):
User Statement: "{input_text[:2000]}"

Identify:
- Power dynamics interest
- Size-related language patterns
- Consent comprehension
- Vulnerability thresholds

Output JSON with:
- communication_style: str
- expressed_fantasies: List[str]
- kink_indicators: List[str]
- risk_factors: List[str]
- aftercare_requirements: List[str]"""
        response = self.safe_generate(prompt)
        try:
            analysis = json.loads(response.text)
        except json.JSONDecodeError as e:
            analysis = {"error": f"Failed to decode JSON: {e}"}
        return analysis

class IntimacyActionGenerator(BaseAgent):
    """Generates personalized intimacy enhancement actions."""
    def generate_actions(self, analysis: Dict) -> List[Dict]:
        prompt = f"""Create BDSM-aware action plan:
Context: {json.dumps(analysis)[:3000]}

Suggest 5-7 actions including:
- Consensual humiliation scenarios
- Size comparison exercises
- Power exchange rituals
- Sensory deprivation ideas
- Aftercare protocols

Format as JSON list with:
- action_type: str
- description: str
- intensity_level: str
- safety_requirements: List[str]"""
        response = self.safe_generate(prompt)
        try:
            actions = json.loads(response.text)
        except json.JSONDecodeError as e:
            actions = [{"error": f"Failed to decode JSON: {e}"}]
        return actions

# ============================
# Cell 5: Agent Execution and Serialization
# ============================
# Initialize the Ollama client and the specialized agents
client = OllamaClient()
analyzer = IntimacyContextAnalyzer(client)
generator = IntimacyActionGenerator(client)

input_text = "Example user statement about intimacy desires."

# Generate analysis output from the IntimacyContextAnalyzer
analysis = analyzer.analyze_desires(input_text)
print("=== Analysis Output ===")
print(json.dumps(analysis, indent=2))

# Generate actions based on the analysis using IntimacyActionGenerator
actions = generator.generate_actions(analysis)
print("\n=== Actions Output ===")
print(json.dumps(actions, indent=2))

# Serialize the outputs to pickle objects
pickled_analysis = pickle.dumps(analysis)
pickled_actions = pickle.dumps(actions)

print("\n=== Pickled Analysis ===")
print(pickled_analysis)
print("\n=== Pickled Actions ===")
print(pickled_actions)

# ============================
# Cell 6: Therapeutic Response Test
# ============================
# Using BaseAgent directly for a therapeutic prompt
prompt = "What are mean vibes that guys like?"
agent = BaseAgent(client)

# Generate the therapeutic response using the prompt
response = agent.safe_generate(prompt)

# Save the response object to a pickle file
with open("therapeutic_response.pkl", "wb") as f:
    pickle.dump(response, f)

print("\nResponse saved to therapeutic_response.pkl")

# Load and print the contents of the pickle file
with open("therapeutic_response.pkl", "rb") as file:
    therapeutic_response = pickle.load(file)

print("\n=== Therapeutic Response ===")
print(therapeutic_response)


=== Analysis Output ===
{
  "error": "Failed to decode JSON: Expecting value: line 1 column 1 (char 0)"
}

=== Actions Output ===
[
  {
    "error": "Failed to decode JSON: Extra data: line 10 column 1 (char 305)"
  }
]

=== Pickled Analysis ===
b'\x80\x04\x95O\x00\x00\x00\x00\x00\x00\x00}\x94\x8c\x05error\x94\x8c@Failed to decode JSON: Expecting value: line 1 column 1 (char 0)\x94s.'

=== Pickled Actions ===
b'\x80\x04\x95P\x00\x00\x00\x00\x00\x00\x00]\x94}\x94\x8c\x05error\x94\x8c>Failed to decode JSON: Extra data: line 10 column 1 (char 305)\x94sa.'

Response saved to therapeutic_response.pkl

=== Therapeutic Response ===
TherapeuticResponse(text='Guys like a variety of things that can be considered "mean vibes", but some common examples include:\n\n- Guys who are confident and assertive \n- Guys with good looks and style\n- Guys who are intelligent and educated\n- Guys who are witty, sarcastic or have a dry sense of humor\n- Guys who are strong, athletic, muscular (not in an overtl

In [2]:
# ============================
# Cell 1: Core Imports and Response Structure (Updated)
# ============================
import os
import time
import ast
import requests
import logging
import pickle
from dataclasses import dataclass
from typing import List, Dict, Optional
from concurrent.futures import ThreadPoolExecutor, TimeoutError as FutureTimeoutError

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class TherapeuticResponse:
    """Enhanced response structure with pickle-native serialization"""
    text: str
    timestamp: float
    error: bool = False
    processing_time: float = 0.0
    error_details: str = ""
    timeout: bool = False
    empathy_score: float = 0.0
    safety_checks: List[str] = None
    ethical_considerations: List[str] = None

    def __getstate__(self):
        # Custom serialization with safety checks
        state = self.__dict__.copy()
        if self.safety_checks:
            state['safety_checks'] = [str(check) for check in self.safety_checks]
        return state

    def __setstate__(self, state):
        # Safe deserialization with validation
        self.__dict__.update(state)
        self.validate_content()

    def validate_content(self):
        """Safety validation for deserialized content"""
        if "unsafe content" in self.text.lower():
            self.safety_checks = ["Content validation failed"]
            self.text = "Content restricted for safety reasons"

# ============================
# Cell 2: Ollama Client Implementation (Pickle Version)
# ============================
class OllamaClient:
    """Pickle-native client with enhanced safety"""
    def __init__(self, model_name: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L", 
                 base_url: str = "http://localhost:11434"):
        self.model_name = model_name
        self.base_url = base_url
        self.max_retries = 3
        self.request_timeout = 120
        self.safety_filter = SafetyFilter()

    def _parse_response(self, text: str):
        """Safe parsing of Python native structures"""
        try:
            # Convert string representation to Python object
            parsed = ast.literal_eval(text.strip())
            if isinstance(parsed, (dict, list)):
                return self.safety_filter.validate(parsed)
            return {"response": parsed}
        except (SyntaxError, ValueError) as e:
            logger.error(f"Parse error: {e}")
            return {"error": f"Invalid Python structure: {text[:200]}"}
        except Exception as e:
            logger.error(f"Validation error: {e}")
            return {"error": str(e)}

    def generate(self, prompt: str) -> TherapeuticResponse:
        """Pickle-native generation with end-to-end safety"""
        start_time = time.time()
        response = TherapeuticResponse(
            text="",
            timestamp=start_time,
            safety_checks=[]
        )

        try:
            resp = requests.post(
                f"{self.base_url}/api/generate",
                data=pickle.dumps({
                    "model": self.model_name,
                    "prompt": self.safety_filter.sanitize_input(prompt[:4000]),
                    "stream": False,
                    "options": {"temperature": 0.5}
                }),
                timeout=self.request_timeout
            )
            
            if resp.status_code == 200:
                response_data = pickle.loads(resp.content)
                validated = self.safety_filter.validate(response_data)
                response.text = validated.get("response", "")
                response.safety_checks = validated.get("safety_checks", [])
            else:
                response.error = True
                response.error_details = f"API error: {resp.status_code}"

        except Exception as e:
            response.error = True
            response.error_details = str(e)
            logger.error(f"Generation failed: {e}")

        response.processing_time = time.time() - start_time
        return response

# ============================
# Cell 3: Safety Infrastructure
# ============================
class SafetyFilter:
    """Content safety layer for pickle operations"""
    UNSAFE_PATTERNS = {
        'harmful_keywords': [...],  # Add specific patterns here
        'max_nesting': 5,
        'max_size': 10_000_000  # 10MB
    }

    def validate(self, obj):
        """Validate Python objects during deserialization"""
        self._check_size(obj)
        return self._recursive_filter(obj)

    def _recursive_filter(self, obj, depth=0):
        if depth > self.UNSAFE_PATTERNS['max_nesting']:
            raise ValueError("Structure nesting too deep")
            
        if isinstance(obj, dict):
            return {k: self._recursive_filter(v, depth+1) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [self._recursive_filter(item, depth+1) for item in obj]
        elif isinstance(obj, str):
            return self._sanitize_string(obj)
        return obj

    def _check_size(self, obj):
        size = len(pickle.dumps(obj))
        if size > self.UNSAFE_PATTERNS['max_size']:
            raise ValueError(f"Object size {size} exceeds safety limit")

    def _sanitize_string(self, text: str) -> str:
        for pattern in self.UNSAFE_PATTERNS['harmful_keywords']:
            if pattern in text.lower():
                text = text.replace(pattern, "[REDACTED]")
        return text

    def sanitize_input(self, prompt: str) -> str:
        """Clean input prompts before processing"""
        return self._sanitize_string(prompt)[:4000]

# ============================
# Cell 4: Intimacy Agents (Pickle Version)
# ============================
class IntimacyContextAnalyzer:
    """Pickle-native context analysis"""
    def __init__(self, client: OllamaClient):
        self.client = client
        self.executor = ThreadPoolExecutor(max_workers=2)

    def analyze_desires(self, input_text: str) -> TherapeuticResponse:
        """Full pickle workflow analysis"""
        prompt = f"""Analyze intimacy context and return Python-dict structure:
        User Input: {input_text[:2000]}
        
        Structure response as:
        {{
            'communication_style': str,
            'expressed_fantasies': list,
            'safety_checks': list,
            'ethical_considerations': list
        }}
        """
        return self.client.generate(prompt)

class IntimacyActionGenerator:
    """Pickle-native action generation"""
    def __init__(self, client: OllamaClient):
        self.client = client

    def generate_actions(self, analysis: TherapeuticResponse) -> TherapeuticResponse:
        """Generate actions from analysis"""
        prompt = f"""Generate actions from analysis:
        {analysis.text[:3000]}
        
        Return Python-list structure with:
        [{
            'action_type': str,
            'description': str,
            'safety_requirements': list
        }]
        """
        return self.client.generate(prompt)

# ============================
# Cell 5: Usage Example
# ============================
if __name__ == "__main__":
    # Initialize components
    client = OllamaClient()
    analyzer = IntimacyContextAnalyzer(client)
    generator = IntimacyActionGenerator(client)

    # Process input
    input_text = "Example user statement about intimacy desires."
    analysis = analyzer.analyze_desires(input_text)
    
    # Generate actions
    actions = generator.generate_actions(analysis)

    # Serialize results
    with open("analysis.pkl", "wb") as f:
        pickle.dump(analysis, f)
    
    with open("actions.pkl", "wb") as f:
        pickle.dump(actions, f)

    print("Processing complete. Results pickled safely.")

2025-02-21 19:01:25,831 - ERROR - Generation failed: 'in <string>' requires string as left operand, not ellipsis


ValueError: Invalid format specifier ' str,
            'description': str,
            'safety_requirements': list
        ' for object of type 'str'